In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>
<br>
<br>
<br>

# **Feature Engineering: bureau**

A Feature Engineering é um processo fundamental para ciência de dados. Ela se refere ao processo de criação e transformação de variáveis para melhorar o desempenho dos modelos de Machine Learning.
<br>
<br>
Nesse notebook vamos trabalhar na criação de variáveis para o dataset **`bureau`**. Esse conjunto de dados fornece dados de crédito de outras instituições financeiras.
<br>
<br>
Com esses dados podemos gerar insights sobre o comportamento passado do cliente em relação aos pagamentos. Isso permite avaliar a pontualidade dos pagamentos, identificar padrões de pagamento e antecipar possíveis dificuldades financeiras, ajudando os credores a mitigar o risco de inadimplência e tomar decisões mais informadas sobre a concessão de crédito.
<br>
<br>
Por se tratar de criação de variáveis, nessa etapa geralmente trabalhamos com muito volume de dados e grande demanda de processamento, por essa razão iremos utilizar o **Spark** e **Spark.SQL** para essa etapa.

<br>

## **Pacotes e Bibliotecas**

In [2]:
# Instalando o PySpark.
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# Importando bibliotecas.
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
import os
import sys

In [4]:
# Configurando a sessão Spark.
spark = SparkSession.builder \
    .appName('app_spark_3') \
    .config('spark.driver.memory', '8g') \
    .config('spark.executor.memory', '4g') \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

<br>
<br>
<br>
<br>

## **Funções**

In [5]:
# Função para substituir espaço por underline e colocar a palavra em letras maiúsculas.
import re

def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()

    return palavra_transformada

<br>
<br>
<br>
<br>

## **Leitura e visualização do conjunto de dados**

In [6]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
dados = spark.read.csv('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/database/bureau.csv', header=True, inferSchema=True)
dados.createOrReplaceTempView('dados')

# Mostra os dados.
dados.show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

In [7]:
# Checando a quantidade de linhas e colunas do DataFrame.

# Quantidade de linhas.
num_rows = dados.count()

# Quantidade de colunas.
num_columns = len(dados.columns)

# Quantidade de IDs únicos.
num_id_curr = dados.select(countDistinct('SK_ID_CURR').alias('num_id_curr')).collect()[0]['num_id_curr']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_CURR" únicos do DataFrame: {num_id_curr}')

Quantidade de linhas do DataFrame: 1716428
Quantidade de colunas do DataFrame: 17
Quantidade de "SK_ID_CURR" únicos do DataFrame: 305811


In [8]:
# Exibindo a estrutura do schema do DataFrame.
dados.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: integer (nullable = true)
 |-- CREDIT_DAY_OVERDUE: integer (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: double (nullable = true)
 |-- DAYS_ENDDATE_FACT: double (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
 |-- CNT_CREDIT_PROLONG: integer (nullable = true)
 |-- AMT_CREDIT_SUM: double (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: integer (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)



In [9]:
# Extraindo os valores únicos da coluna 'CREDIT_ACTIVE'.
status = dados.select('CREDIT_ACTIVE').distinct().rdd.flatMap(lambda x: x).collect()

# Aplicando a função de tratamento de palavra a cada valor único.
credit_active_list = [tratamento_palavra(status) for status in status]

# Exibindo a lista resultante.
print(credit_active_list)

['BAD_DEBT', 'SOLD', 'ACTIVE', 'CLOSED']


<br>
<br>
<br>
<br>

## **Criação de flags**

A criação de flags binárias 1 ou 0 será útil para nos auxiliar na visão temporal dos dados e na criação das variáveis.
<br>
<br>
Serão criadas flags para os últimos 3, 6, 12, 24 e 36 meses.

### **Flags de janela de tempo**

In [10]:
df_temp_01 = spark.sql('''

SELECT
  *,
  CASE
    WHEN DAYS_CREDIT >=   -90 THEN 1 ELSE 0 END AS U3M,
  CASE
    WHEN DAYS_CREDIT >=  -180 THEN 1 ELSE 0 END AS U6M,
  CASE
    WHEN DAYS_CREDIT >=  -360 THEN 1 ELSE 0 END AS U12M,
  CASE
    WHEN DAYS_CREDIT >=  -720 THEN 1 ELSE 0 END AS U24M,
  CASE
    WHEN DAYS_CREDIT >= -1080 THEN 1 ELSE 0 END AS U36M
FROM
  dados
ORDER BY
  `SK_ID_BUREAU`;

''')
df_temp_01.createOrReplaceTempView('df_temp_01')
df_temp_01.show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+--------------------+------------------+-----------+---+---+----+----+----+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|         CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|U3M|U6M|U12M|U24M|U36M|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+--------------------+------------------+-----------+---+---+----+----+----+
|    166497|     5000000|       Close

<br>
<br>
<br>
<br>

## **Criação de variáveis explicativas**

Vamos criar as variáveis explicativas com base nas variáveis já existentes, usando a sumarização para as janelas temporais criadas na etapa anterior.

In [11]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = ['CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
                           'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M', 'U24M', 'U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for cat1 in credit_active_list:
    for coluna in colunas_agregacao_total:
      for flag in colunas_flags:
        if 'DAY' in coluna or 'CNT' in coluna:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_{cat1}'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_{cat1}'))
          expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MAX_{coluna.upper()}_{flag.upper()}_{cat1}'))
          expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MIN_{coluna.upper()}_{flag.upper()}_{cat1}'))
        else:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_{cat1}'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_{cat1}'))
          expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MAX_{coluna.upper()}_{flag.upper()}_{cat1}'))
          expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MIN_{coluna.upper()}_{flag.upper()}_{cat1}'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_02 = df_temp_01.groupBy('SK_ID_BUREAU').agg(*expressoes_agregacao).orderBy('SK_ID_BUREAU')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_02.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_02.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_02.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_02.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_02.show(20, False)

Quantidade Total de Variáveis Criadas: 880
Nomes das Variáveis Criadas: ['QT_TOT_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT', 'QT_MED_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT', 'QT_MAX_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT', 'QT_MIN_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT', 'QT_TOT_CREDIT_DAY_OVERDUE_U6M_BAD_DEBT', 'QT_MED_CREDIT_DAY_OVERDUE_U6M_BAD_DEBT', 'QT_MAX_CREDIT_DAY_OVERDUE_U6M_BAD_DEBT', 'QT_MIN_CREDIT_DAY_OVERDUE_U6M_BAD_DEBT', 'QT_TOT_CREDIT_DAY_OVERDUE_U12M_BAD_DEBT', 'QT_MED_CREDIT_DAY_OVERDUE_U12M_BAD_DEBT', 'QT_MAX_CREDIT_DAY_OVERDUE_U12M_BAD_DEBT', 'QT_MIN_CREDIT_DAY_OVERDUE_U12M_BAD_DEBT', 'QT_TOT_CREDIT_DAY_OVERDUE_U24M_BAD_DEBT', 'QT_MED_CREDIT_DAY_OVERDUE_U24M_BAD_DEBT', 'QT_MAX_CREDIT_DAY_OVERDUE_U24M_BAD_DEBT', 'QT_MIN_CREDIT_DAY_OVERDUE_U24M_BAD_DEBT', 'QT_TOT_CREDIT_DAY_OVERDUE_U36M_BAD_DEBT', 'QT_MED_CREDIT_DAY_OVERDUE_U36M_BAD_DEBT', 'QT_MAX_CREDIT_DAY_OVERDUE_U36M_BAD_DEBT', 'QT_MIN_CREDIT_DAY_OVERDUE_U36M_BAD_DEBT', 'QT_TOT_DAYS_CREDIT_ENDDATE_U3M_BAD_DEBT', 'QT_MED_DAYS_CREDIT_ENDDATE_U3M_

<br>
<br>
<br>
<br>

## **Join das variáveis criadas com o book de variáveis da Bureau Balance**

In [12]:
# Lendo a dataset com o book de variáveis criadas a partir da Bureau Balance.
bureau_balance = spark.read.parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_bureau_balance')

In [13]:
# Selecionando todas as colunas de book_bureau_balance, exceto 'PK_DATREF' e 'PK_DAT_PROC'.
bureau_balance_cols = [col for col in bureau_balance.columns if col not in ['PK_DATREF', 'PK_DAT_PROC']]

# Realizando o join excluindo as colunas 'PK_DATREF' e 'PK_DAT_PROC'.
df_temp_03 = df_temp_02.join(bureau_balance.select(*bureau_balance_cols), on='SK_ID_BUREAU', how='left')

# Mostrando o resultado do join.
df_temp_03.show(20, False)

+------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+----------------------------------

<br>
<br>
<br>
<br>

## **Trazendo o ID 'SK_ID_CURR' para o book criado**

In [14]:
df_temp_04 = df_temp_01.select('SK_ID_BUREAU', 'SK_ID_CURR').join(df_temp_03, on='SK_ID_BUREAU', how='left')

# Mostrar o resultado do join
df_temp_04.show(20, False)

+------------+----------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+-----------------------

<br>
<br>
<br>
<br>

## **Sumarização dos dados pelo ID 'SK_ID_CURR'**

Vamos fazer a agregação agora pelo ID **`SK_ID_CURR`**, que é o mesmo ID encontrado na tabela de **application_train**.
<br>
<br>
Farei em duas partes para otimizar o tempo de processamento de cada etapa.

### **Parte 1**

In [15]:
# Definindo as colunas para agregação.
colunas_agregacao_total = df_temp_04.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_BUREAU')

expressoes_agregacao = []

for coluna in colunas_agregacao_total:
  if 'DAY' in coluna or 'CNT' in coluna or 'STATUS' in coluna:
    expressoes_agregacao.append(round(sum(col(coluna)), 2).alias(f'QT_TOT_{coluna.upper()}_BUREAU'))
    expressoes_agregacao.append(round(avg(col(coluna)), 2).alias(f'QT_MED_{coluna.upper()}_BUREAU'))
    expressoes_agregacao.append(round(max(col(coluna)), 2).alias(f'QT_MAX_{coluna.upper()}_BUREAU'))
    expressoes_agregacao.append(round(min(col(coluna)), 2).alias(f'QT_MIN_{coluna.upper()}_BUREAU'))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_05 = df_temp_04.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_05.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_05.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_05.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_05.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

Quantidade Total de Variáveis Criadas: 1760
Nomes das Variáveis Criadas: ['QT_TOT_QT_TOT_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MED_QT_TOT_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MAX_QT_TOT_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MIN_QT_TOT_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_TOT_QT_MED_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MED_QT_MED_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MAX_QT_MED_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MIN_QT_MED_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_TOT_QT_MAX_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MED_QT_MAX_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MAX_QT_MAX_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MIN_QT_MAX_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_TOT_QT_MIN_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MED_QT_MIN_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MAX_QT_MIN_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_MIN_QT_MIN_CREDIT_DAY_OVERDUE_U3M_BAD_DEBT_BUREAU', 'QT_TOT_QT_TOT_CREDIT_DAY_OVER

In [16]:
# Mostrando o novo DataFrame com as variáveis criadas.
#df_temp_05.show(20, False)

In [18]:
# Salvando a tabela sumarizada.
df_temp_05.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_bureau_01')

<br>

### **Parte 2**

In [19]:
# Definindo as colunas para agregação.
colunas_agregacao_total = df_temp_04.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_BUREAU')

expressoes_agregacao = []

for coluna in colunas_agregacao_total:
  if 'DAY' not in coluna and 'CNT' not in coluna and 'STATUS' not in coluna:
    expressoes_agregacao.append(round(sum(col(coluna)), 2).alias(f'VL_TOT_{coluna.upper()}_BUREAU'))
    expressoes_agregacao.append(round(avg(col(coluna)), 2).alias(f'VL_MED_{coluna.upper()}_BUREAU'))
    expressoes_agregacao.append(round(max(col(coluna)), 2).alias(f'VL_MAX_{coluna.upper()}_BUREAU'))
    expressoes_agregacao.append(round(min(col(coluna)), 2).alias(f'VL_MIN_{coluna.upper()}_BUREAU'))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_06 = df_temp_04.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_06.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_06.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_06.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_06.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

Quantidade Total de Variáveis Criadas: 1920
Nomes das Variáveis Criadas: ['VL_TOT_VL_TOT_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MED_VL_TOT_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MAX_VL_TOT_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MIN_VL_TOT_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_TOT_VL_MED_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MED_VL_MED_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MAX_VL_MED_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MIN_VL_MED_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_TOT_VL_MAX_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MED_VL_MAX_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MAX_VL_MAX_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MIN_VL_MAX_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_TOT_VL_MIN_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MED_VL_MIN_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MAX_VL_MIN_AMT_CREDIT_MAX_OVERDUE_U3M_BAD_DEBT_BUREAU', 'VL_MIN_VL_MIN_AMT_CREDIT_

In [ ]:
# Mostrando o novo DataFrame com as variáveis criadas.
#df_temp_06.show(20, False)

In [20]:
# Salvando a tabela sumarizada.
df_temp_06.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_bureau_02')